В этой работе мы реализуем переводчик с русского языка на язык Старшей Речи и наоборот. Тоже самое с английским.
Мы реализуем алгоритм для перевода с русского на Старшую Речь.
Далее используем его для формирования примерно 200 эталонных переводов, на основе которых будем оценивать качество переводов следующих моделей:
1. Llama
2. Deepseek
3. Gemma

Перед началом переводов каждая модель будет настроена промптами.

In [1]:
import pandas as pd

import requests
import re
import string
import json

import pymorphy2
lemmer = pymorphy2.MorphAnalyzer()

import ollama
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [2]:
df = pd.read_csv('../lr3/witcher_words.csv')

Напишем прямой алгоритм перевода с русского на Старшую Речь.

In [3]:
def translate_russian_to_elvish(text: str) -> str:
    """
    Переводит русский текст на язык Старшей Речи.
    Слова, перевод которым подобрать не получилось, латинизируются.
    Присутствует обработка некоторых устойчивых выражений из словаря.
    """

    # Унифицируем е/ё:
    text = text.replace('ё', 'е')

    # Результирующие подстроки для склеивания в конце.
    result_subs = []

    # Отделяем слова от пунктуационных знаков:
    subs = re.split(r'(?<=\.|!|\?|,|;|:|\s|\n|\(|\))|(?=\.|!|\?|,|;|:|\s|\n|\(|\))', text)

    next_i = 0
    for i, sub in enumerate(subs):
        if i < next_i:
            continue

        next_i += 1
        if sub in string.punctuation + ' \n':
            result_subs.append(sub)
            continue

        word = sub
        is_title = word.replace('\'', '').istitle()
        word = word.lower()
        word = _word_to_lemm(word)

        # Текущее слово может быть началом какого-то устойчивого выражения, проверим это.
        it_is_real_phrase = False
        phrases = _find_phrases_that_starts_with_russian_word(word)
        if phrases and i != len(subs) - 1:
            # Будем прибавлять сюда последующие пробелы и слова, пока не встретим что-то лишнее либо не убедимся
            # в том, что это выражение - действительная устойчивая фраза из словаря.
            summ_word = word

            # Количество итераций для пропуска во внешнем цикле (через `next_i`)
            count_to_pass = 0

            for sub in subs[i + 1:]:
                if sub in '.!?':
                    break

                count_to_pass += 1
                if sub not in string.punctuation + ' ':
                    sub = _word_to_lemm(sub.lower())
                elif sub == ',':
                    # Опускаем запятые между словами потенциальной фразы.
                    sub = ''

                summ_word += sub

                need_continue = False
                if summ_word in phrases:
                    it_is_real_phrase = True
                    break

                for phrase in phrases:
                    if summ_word in phrase:
                        need_continue = True
                        break

                if not need_continue:
                    break

            if it_is_real_phrase:
                word = summ_word
                next_i += count_to_pass
                translation = _try_to_find_russian_word_translation(word)

        if not it_is_real_phrase:
            # Стандартная обработка одиночного слова с подбором синонимов через сайт.
            translation = _translate_russian_word(word, _synonyms_of_word_from_web_api)

        if is_title:
            translation = translation.capitalize()

        result_subs.append(translation)

    return ''.join(result_subs)


def _word_to_lemm(word: str) -> str:
    """Переводит слово/словосочетание в начальную форму."""
    parts = []
    for part in word.split():
        part = lemmer.parse(part)[0].normal_form.replace('ё', 'е')
        parts.append(part)

    return ' '.join(parts)


def _translate_russian_word(word: str, synonyms_finding_func) -> str:
    """
    Переводит русское слово/словосочетание на Старшую Речь.
    При отсутствии слова в датасете использует перебор синонимов из функции `synonyms_finding_func`.
    Если последнее так же не помогло, то делает латинизацию слова.
    """
    translation = None
    try:
        translation = _try_to_find_russian_word_translation(word)
    except ValueError:
        for synonym in _synonyms_of_word_from_web_api(word):
            try:
                translation = _try_to_find_russian_word_translation(synonym)
            except ValueError:
                continue

    if translation is None:
        translation = _transliterate_russian_to_latin(word)

    return translation


def _try_to_find_russian_word_translation(word: str) -> str:
    """Ищет перевод русского слова/словосочетания в датасете."""
    result = df.loc[df['translation'] == word, 'text']
    if not result.empty:
        return result.values[0]
    else:
        raise ValueError


def _find_phrases_that_starts_with_russian_word(word: str) -> list[str]:
    return df.loc[df['translation'].str.startswith(word + ' '), 'translation'].tolist()


def _synonyms_of_word_from_web_api(word: str) -> str:
    """Выдаёт всевозможные синонимы заданного слова. Использует сетевой запрос."""
    html = requests.get(f'https://text.ru/synonym/{word}').text
    
    match = re.search(r'<meta name=\"description\" content=\"Синонимы к слову [^—]*:([^\"]+)\" />', html)
    if match is None:
        return []

    text = match.group(1)
    synonyms = [synonym.strip() for synonym in text.strip().split('—')]
    synonyms.remove('')

    return synonyms


def _transliterate_russian_to_latin(russian_word: str) -> str:
    """Функция для латинизации русского слова."""
    translit_dict = {
        'а': 'a', 
        'б': 'b', 
        'в': 'v', 
        'г': 'g', 
        'д': 'd', 
        'е': 'e', 
        'ё': 'yo',
        'ж': 'zh', 
        'з': 'z', 
        'и': 'i', 
        'й': 'y', 
        'к': 'k', 
        'л': 'l', 
        'м': 'm',
        'н': 'n', 
        'о': 'o', 
        'п': 'p', 
        'р': 'r', 
        'с': 's', 
        'т': 't', 
        'у': 'u',
        'ф': 'f', 
        'х': 'kh', 
        'ц': 'ts', 
        'ч': 'ch', 
        'ш': 'sh', 
        'щ': 'shch', 
        'ъ': '',
        'ы': 'y', 
        'ь': '', 
        'э': 'e', 
        'ю': 'yu', 
        'я': 'ya',
    }

    latin_word = ''.join(translit_dict.get(symbol, symbol) for symbol in russian_word.lower())
    return latin_word

Посмотрим пример:

In [4]:
print(translate_russian_to_elvish('Здравствуй, Геральт Белый Волк! Твоё присутствие - большая честь для всех нас.'))

Cead, Geralt Gwynbleidd! Do prisutstvie - bolshiy aere а evellienn sinn.


Сформируем списки эталонных переводов. Запускать следующую ячейку, если файла "backup.json" нет. Если есть, то вторую, после неё.

In [15]:
sequences = []
with open('./sequences.txt', 'r', encoding='utf-8') as f:
    i = 0
    for record in f:
        record = record.strip()
        if not record:
            continue

        if i % 2 == 0:
            # Записываем русское предложение + перевод.
            sequences.append([
                record,
                translate_russian_to_elvish(record),
            ])
        else:
            # Дозаписываем соответствующее английское предложение.
            sequences[-1].append(record)

        i += 1
        if i % 50 == 0:
            print('Часть записей обработана.')

# Сохраняем эталонные переводы, чтобы не проводить эту процедуру каждый раз.
with open('backup.json', 'w', encoding='utf-8') as f:
    json.dump(sequences, f)

Часть записей обработана.
Часть записей обработана.
Часть записей обработана.
Часть записей обработана.
Часть записей обработана.
Часть записей обработана.
Часть записей обработана.
Часть записей обработана.


In [5]:
with open('backup.json', 'r', encoding='utf-8') as f:
    sequences = json.load(f)

Рассмотрим модели LLM для решения задачи перевода. Напишем общий класс для всех наших моделей из Olama.

In [58]:
class Translator:
    """
    Обёртка над моделью LLM из Olama для реализации перевода.
    Реализованы двухсторонние переводы с русского/английского на Старшую речь.
    Поступаемый текст сначала токенизируется, лемматизируется, потом происходит поиск используемых слов
    по словарю.
    После всего этого исходный текст (всегда русский либо эльфский) и собранная часть словаря отправляются модели.
    Это ускорило обработку во много раз, поскольку изначально модели залился словарь целиком (это был этап "инициализации"),
    а потом происходили переводы.
    Здесь нам даже не нужно хранить историю предыдущих сообщений.
    """

    _RESULT_LIMITER: str = 'RESULT'
    _SYNONYM_SEPARATOR: str = ','

    def __init__(self, model_name: str) -> None:
        self._model_name = model_name

    def translate_russian_to_elvish(self, text: str) -> str:
        """Переводит русский текст на Старшую Речь."""
        dictionary = self._make_dictionary_by_russian_text(text)
        return self._request(f'Переведи русский текст "{text}" на другой язык согласно словарю:\n' + dictionary)

    def translate_elvish_to_russian(self, text: str) -> str:
        """Переводит текст на языке Старшей Речи на русский."""
        dictionary = self._make_dictionary_by_elvish_text(text)
        return self._request(f'Переведи вымышленный текст "{text}" на русский согласно словарю:\n' + dictionary)

    def translate_english_to_elvish(self, text: str) -> str:
        """Переводит английский текст на Старшую Речь."""
        text = self._request(f'Переведи английский текст "{text}" на русский.')
        return self.translate_russian_to_elvish(text)

    def translate_elvish_to_english(self, text: str) -> str:
        """Переводит текст на языке Старшей Речи на английский."""
        text = self.translate_elvish_to_russian(text)
        return self._request(f'Переведи русский текст "{text}" на английский.')

    def _make_dictionary_by_russian_text(self, text: str) -> str:
        """Формирует текстовый словарь, опираясь на слова используемые в русском тексте."""
        dictionary: str = ''

        words = self._extract_lemms(text)
        for word in words:
            dictionary += word + ' - ' + _translate_russian_word(word, self._synonyms_of_word) + '\n'
            for translation, phrase in self._russian_phrases_that_starts_with_word_and_translations(word):
                dictionary += phrase + ' - ' + translation + '\n'

        return dictionary

    def _make_dictionary_by_elvish_text(self, text: str) -> str:
        """Формирует текстовый словарь, опираясь на слова используемые в тексте на языке Старшей Речи."""
        dictionary: str = ''

        words = self._extract_words(text)
        for word in words:
            for translation in self._elvish_word_translations(word):
                dictionary += word + ' - ' + translation + '\n'
            for phrase, translation in self._elvish_phrases_that_starts_with_word_and_translations(word):
                dictionary += phrase + ' - ' + translation + '\n'

        return dictionary

    def _extract_lemms(self, text: str) -> list[str]:
        """Извлекает из текста слова и приводит их к начальной форме."""
        return [_word_to_lemm(word) for word in self._extract_words(text)]

    def _extract_words(self, text: str) -> list[str]:
        """Извлекает из текста слова и переводит их в нижний регистр."""
        text = self._remove_punctuation(text)
        return text.lower().split()

    @staticmethod
    def _remove_punctuation(text: str) -> str:
        """Удаляет из текста все знаки препинания и пунктуации."""
        return text.translate(
            str.maketrans('', '', string.punctuation),
        )

    def _synonyms_of_word(self, word: str) -> list[str]:
        """Выдаёт синонимы слова, которые подобрала модель."""
        text = self._request(f'Подбери синонимы слова "{word}". Разделяй их через {self._SYNONYM_SEPARATOR}.')
        return [word.strip() for word in text.split(self._SYNONYM_SEPARATOR)]

    def _russian_phrases_that_starts_with_word_and_translations(self, word: str) -> list[tuple[str, str]]:
        """
        Выдает все фразы и их переводы, которые начинаются с переданного русского слова.
        Первое в каждом кортеже эльфское, второе - русское.
        """
        return self._phrases_that_starts_with_word_and_translations(word, 'translation')

    def _elvish_phrases_that_starts_with_word_and_translations(self, word: str) -> list[tuple[str, str]]:
        """
        Выдает все фразы и их переводы, которые начинаются с переданного слова на Старшей Речи.
        Первое в каждом кортеже эльфское, второе - русское.
        """
        return self._phrases_that_starts_with_word_and_translations(word, 'text')

    @staticmethod
    def _phrases_that_starts_with_word_and_translations(word: str, col: str) -> list[tuple[str, str]]:
        """
        Выдает все фразы и их переводы, которые начинаются с переданного слова в заданном столбце `df`.
        Первое в каждом кортеже эльфское, второе - русское.
        """
        return df[df[col].str.startswith(word + ' ')].itertuples(index=False)

    @staticmethod
    def _elvish_word_translations(word: str) -> list[str]:
        """Выдает всевозможные переводы слова на Старшей Речи."""
        result = df.loc[df['text'] == word, 'translation']
        return result.values

    def _request(self, text: str, extract_result: bool = True) -> str:
        """Отправляет запрос модели и выдает ответ."""
        if extract_result:
            text += f'\nСлева и справа от результата напиши "{self._RESULT_LIMITER}". Больше ничего лишнего не пиши.'

        answer = ollama.chat(model=self._model_name, messages=[{
            'role': 'user', 
            'content': text,
        }])['message']['content']

        # DeepSeek thinking isn't needed :)
        if '</think>' in answer:
            answer = answer.split('</think>')[1]

        answer = answer.split(self._RESULT_LIMITER)[1]
        return answer.strip()

Посмотрим, что получилось:

In [57]:
t = Translator('gemma3')
print(t.translate_elvish_to_english('Aé lyubit gotovit noel ite.'))

I love cooking new food.


Начнем тестировать наши модели. Используем метрику BLEU.

In [59]:
def _test_model(model_name: str) -> None:
    print(model_name)
    translator = Translator(model_name)

    bleu_scores_russian_to_elvish = []
    bleu_scores_elvish_to_russian = []

    bleu_scores_english_to_elvish = []
    bleu_scores_elvish_to_english = []

    for i, record in enumerate(sequences, start=1):
        if i % 50 == 0:
            print('Часть записей обработана.')

        try:
            bleu_scores_russian_to_elvish.append(
                sentence_bleu(
                    translator.translate_russian_to_elvish(record[0]), 
                    record[1],
                )
            )
        except Exception as e:
            print(e)
            continue

        try:
            bleu_scores_elvish_to_russian.append(
                sentence_bleu(
                    translator.translate_elvish_to_russian(record[1]), 
                    record[0],
                )
            )
        except Exception as e:
            print(e)
            continue

        try:
            bleu_scores_english_to_elvish.append(
                sentence_bleu(
                    translator.translate_english_to_elvish(record[2]), 
                    record[1],
                )
            )
        except Exception as e:
            print(e)
            continue

        try:
            bleu_scores_elvish_to_english.append(
                sentence_bleu(
                    translator.translate_elvish_to_english(record[1]), 
                    record[2],
                )
            )
        except Exception as e:
            print(e)
            continue

    score = sum(bleu_scores_russian_to_elvish) / len(bleu_scores_russian_to_elvish)
    print('Russian-To-Elvish Score:', score)

    score = sum(bleu_scores_elvish_to_russian) / len(bleu_scores_elvish_to_russian)
    print('Elvish-To-Russian Score:', score)

    score = sum(bleu_scores_english_to_elvish) / len(bleu_scores_english_to_elvish)
    print('English-To-Elvish Score:', score)

    score = sum(bleu_scores_elvish_to_english) / len(bleu_scores_elvish_to_english)
    print('Elvish-To-English Score:', score)

In [61]:
_test_model('llama3')

llama3
('А',)
Часть записей обработана.
Часть записей обработана.
Часть записей обработана.
('S',)
('S',)
Часть записей обработана.
Russian-To-Elvish Score: 1.4081319323328796e-231
Elvish-To-Russian Score: 1.3979871551804934e-231
English-To-Elvish Score: 1.3821330859369786e-231
Elvish-To-English Score: 1.370886268179268e-231


In [ ]:
_test_model('deepseek-r1')

deepseek-r1
('M',)
list index out of range
list index out of range
list index out of range
('T',)
('I',)


In [60]:
_test_model('gemma3')

gemma3
('A',)
Часть записей обработана.
Часть записей обработана.
Часть записей обработана.
Часть записей обработана.
('I',)
Russian-To-Elvish Score: 1.3956151220093285e-231
Elvish-To-Russian Score: 1.3790497055025017e-231
English-To-Elvish Score: 1.3802689495482624e-231
Elvish-To-English Score: 1.3533990972923734e-231


Вывод:
llama3 работает лучше всего по метрикам и выдает неплохие ответы.
deepseek-r1 работает очень медленно и выдаёт кривые ответы.
gemma3 работает хуже по метрикам, чем llama3 (из-за того, что пропускает пунктуацию), но зато довольно быстро.